In [13]:
import pandas as pd
from numpy import nan
import os
import requests

from tier1_to_dcp_dict import tier1_to_dcp

In [14]:
collection_id = 'bcb61471-2a44-4d00-a0af-ff085512674c'
dataset_id = '0b75c598-0893-4216-afe8-5414cab7739d'

In [15]:
study_metadata = pd.read_csv(f"metadata/{collection_id}_{dataset_id}_study_metadata.csv", header=None).T
study_metadata.columns = study_metadata.iloc[0]
study_metadata.drop(0, axis=0, inplace=True)
sample_metadata = pd.read_csv(f"metadata/{collection_id}_{dataset_id}_metadata.csv")

In [16]:
hca_template_url = 'https://github.com/ebi-ait/geo_to_hca/raw/master/template/hca_template.xlsx'
hca_template_file = requests.get(hca_template_url)
dcp_spreadsheet = pd.read_excel(hca_template_file.content, sheet_name=None, skiprows= [0,1,2,4])

# save the 4-row header of the original spreadsheet with programmatic name as column names
dcp_headers = pd.read_excel(hca_template_file.content, sheet_name=None, header=None)
for tab in dcp_headers:
    dcp_headers[tab].rename(columns=dcp_headers[tab].iloc[3], inplace= True)

In [17]:
dcp_spreadsheet['Project - Publications'] = pd.DataFrame({key: \
    (study_metadata['doi'].tolist() if key.endswith("doi") \
        else [nan]) \
        for key in dcp_spreadsheet['Project - Publications'].keys()})

In [18]:
def reformat_column_values(column_name, value):
    if value == "Yes":
        return column_name
    else:
        return nan

for column in sample_metadata.columns:
    if "diabetes" in column.lower() or "hypertension" in column.lower():
        sample_metadata[column] = sample_metadata.apply(lambda row: reformat_column_values(column, row[column]), axis=1)

sample_dcp_fields = sample_metadata.rename(columns=tier1_to_dcp)

In [19]:
for tab in dcp_spreadsheet:
    keys_union = [key for key in dcp_spreadsheet[tab].keys() if key in sample_dcp_fields.keys()]
    # if tab contains only the input biomaterial name, then skip the tab
    if (len(keys_union) == 1) and (tab.lower().replace(" ", "_") != keys_union[0].split(".")[0]):
        continue
    # collapse arrays in duplicated columns
    if any(sample_dcp_fields[keys_union].columns.duplicated()):
        for dub_cols in set(sample_dcp_fields[keys_union].columns[sample_dcp_fields[keys_union].columns.duplicated()]):
            df = sample_dcp_fields[dub_cols]
            sample_dcp_fields.drop(columns=dub_cols, inplace=True)
            sample_dcp_fields[dub_cols] = df[dub_cols].apply(lambda x: '||'.join(x.dropna().astype(str)),axis=1)

    dcp_spreadsheet[tab] = pd.concat([dcp_spreadsheet[tab],sample_dcp_fields[keys_union]])
    dcp_spreadsheet[tab].dropna(how='all', inplace=True)

In [20]:
with pd.ExcelWriter(f"metadata/{collection_id}_{dataset_id}_dcp.xlsx") as writer:
    for tab in dcp_spreadsheet:
        if not dcp_spreadsheet[tab].empty:
            print(tab)
            pd.concat([dcp_headers[tab], dcp_spreadsheet[tab]]).to_excel(writer, sheet_name=tab, index=False, header=False)


Project - Publications
Donor organism
Specimen from organism
Cell suspension
Library preparation protocol


In [22]:
dcp_spreadsheet

{'Project': Empty DataFrame
 Columns: [project.project_core.project_short_name, project.project_core.project_title, project.project_core.project_description, project.supplementary_links, project.insdc_project_accessions, project.geo_series_accessions, project.array_express_accessions, project.insdc_study_accessions, project.biostudies_accessions, project.ega_accessions, project.dbgap_accessions, project.estimated_cell_count, project.data_use_restriction, project.duos_id, project.hca_bionetworks.name, project.hca_bionetworks.hca_tissue_atlas, project.hca_bionetworks.hca_tissue_atlas_version, project.hca_bionetworks.atlas_project]
 Index: [],
 'Project - Contributors': Empty DataFrame
 Columns: [project.contributors.name, project.contributors.email, project.contributors.phone, project.contributors.institution, project.contributors.laboratory, project.contributors.address, project.contributors.country, project.contributors.corresponding_contributor, project.contributors.project_role.text,